In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
loans_binary_encoded = pd.get_dummies(df, columns=["home_ownership",
                                                   "verification_status",
                                                   "hardship_flag",
                                                   "debt_settlement_flag",
                                                   "pymnt_plan",
                                                   "initial_list_status",
                                                   "application_type",
                                                   "next_pymnt_d",
                                                   "issue_d"])
loans_binary_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,0,1,1,0,0,1,0,0,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,1,1,0,0,1,0,0,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,1,1,0,0,1,0,0,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,0,1,1,0,0,1,0,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,0,1,1,0,0,1,0,0,1


# Split the Data into Training and Testing

In [7]:
# Create our features
X = loans_binary_encoded.drop(columns='loan_status')

# Create our target
y = df['loan_status']

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,1.0,0.123879,0.876121,0.860340,0.139660,0.383161,0.616839,0.371696,0.451066,0.177238
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.0,0.329446,0.329446,0.346637,0.346637,0.486161,0.486161,0.483261,0.497603,0.381873
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.0,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.0,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values
Counter(y)

Counter({'low_risk': 68470, 'high_risk': 347})

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape 

(51612, 95)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [11]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

blf = BalancedRandomForestClassifier(random_state=1, n_estimators=100).fit(X_train_scaled, y_train)

In [12]:
# Calculated the balanced accuracy score

y_pred = blf.predict(X_test)

from sklearn.metrics import accuracy_score
print(f"balanced accuracy score: {accuracy_score(y_test, y_pred)}")

balanced accuracy score: 0.9904678872420808


In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm_df=pd.DataFrame(cm, index=["Actual High-Risk", "Actual Low-Risk"],columns=["predicted High-Risk","predicted Low-Risk"])
cm_df

,predicted High-Risk,predicted Low-Risk
Actual High-Risk,3,101
Actual Low-Risk,63,17038


In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.05      0.03      1.00      0.04      0.17      0.03       104
   low_risk       0.99      1.00      0.03      1.00      0.17      0.03     17101

avg / total       0.99      0.99      0.03      0.99      0.17      0.03     17205



In [16]:
# List the features sorted in descending order by feature importance
importances = blf.feature_importances_
sorted(zip(importances, X.columns), reverse=True)

[(0.07962160975849848, 'total_rec_prncp'),
 (0.06420558965777451, 'last_pymnt_amnt'),
 (0.05815917708813084, 'total_pymnt_inv'),
 (0.05719474761616686, 'total_pymnt'),
 (0.05271406154290848, 'total_rec_int'),
 (0.03027023507237747, 'int_rate'),
 (0.020838978605293934, 'max_bal_bc'),
 (0.019623287030965365, 'issue_d_Jan-2019'),
 (0.019616459579850788, 'installment'),
 (0.017915793942590522, 'out_prncp'),
 (0.017223798079315976, 'dti'),
 (0.01625071106831437, 'bc_util'),
 (0.016069638572924906, 'mo_sin_old_rev_tl_op'),
 (0.015917850209230377, 'tot_hi_cred_lim'),
 (0.01541809142766988, 'annual_inc'),
 (0.015235950801496695, 'bc_open_to_buy'),
 (0.015180220093664633, 'avg_cur_bal'),
 (0.014954649301671574, 'revol_bal'),
 (0.01461380816482186, 'tot_cur_bal'),
 (0.014225533395755587, 'loan_amnt'),
 (0.014053058982140789, 'il_util'),
 (0.013911167318281802, 'out_prncp_inv'),
 (0.01350739175327045, 'total_bal_ex_mort'),
 (0.013222047127096863, 'total_bal_il'),
 (0.01320820000709989, 'issue_d_M

### Easy Ensemble AdaBoost Classifier

In [20]:
# Train the EasyEnsembleClassifier

from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(random_state=1, n_estimators=100)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [21]:
# Calculated the accuracy score

y_pred2 = blf.predict(X_test)

from sklearn.metrics import accuracy_score
print(f"balanced accuracy score: {accuracy_score(y_test, y_pred2)}")

balanced accuracy score: 0.9904678872420808


In [22]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred2)
cm_df=pd.DataFrame(cm, index=["Actual High-Risk", "Actual Low-Risk"],columns=["predicted High-Risk","predicted Low-Risk"])
cm_df

,predicted High-Risk,predicted Low-Risk
Actual High-Risk,3,101
Actual Low-Risk,63,17038


In [23]:
# Print the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred2))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.05      0.03      1.00      0.04      0.17      0.03       104
   low_risk       0.99      1.00      0.03      1.00      0.17      0.03     17101

avg / total       0.99      0.99      0.03      0.99      0.17      0.03     17205



In [24]:
# Print the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred2))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.05      0.03      1.00      0.04      0.17      0.03       104
   low_risk       0.99      1.00      0.03      1.00      0.17      0.03     17101

avg / total       0.99      0.99      0.03      0.99      0.17      0.03     17205

